We have tried building a spam detection filter using Python and Spark. Our data set consists of volunteered text messages from a study in Singapore and some spam texts from a UK reporting site. 

A spam is an irrelevant or inappropriate message sent on internet to a large group of people. A Ham, on the other hand, is a message which is not a spam and is important to us. 
We aim to segregate these two messages and classify them as accurately as possible.

In [0]:
from pyspark.sql import SparkSession

In [0]:
#create a spark session with a relevant name
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
#read the dataset
data = spark.read.csv('dbfs:/FileStore/shared_uploads/sanchari.gautam@gmail.com/SMSSpamCollection',inferSchema=True,sep="\t")
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

As we can see, there is no column name present in our dataset, we would be renaming them as class and text.

In [0]:
#rename the column names
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

Now, we will be tokenizing each text message into words called tokens. Tokenizer helps us build the models to perform the clsssification. Raw texts are hard to be interpreted and hence splitting them up into smaller units helps us understand the context of the message. 

First, let us see if there is any pattern that we could find out by a simple analysis. We would be trying to find out the average length of spam and ham text messages.
Let us find out how to do that!

In [0]:
from pyspark.sql.functions import length

In [0]:
#create a new column "length" containing the length of each text message
data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
#computing the average length of spam and ham messages
data.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover,CountVectorizer, 
                                IDF, StringIndexer)

In [0]:
#create an object of the Tokenizer to convert text column into tokens
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')

Stopwords are the most common words in any natural language. These words do not add any value to the over all meaning and context of the document. Hence, it is better to remove these distracting words in order to make the performance of our model better. 

Some examples of the stopwords are "the, is, are, with, and, etc."

In [0]:
#creating an object of the StopWordsRemover to remove stopwords from the column token_text
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')

Now, after tokenizing the texts and removing the stopwords, we are only left with those tokens which are important and relevant to our aim of classifying them into spam and ham. 

Next, we will try to find out the term frequency of each term or token in the text with the help of CountVectorizer class. Let us see how to do that!

In [0]:
#finding the term frequency of each token 
countvec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')

Inverse Document Frequency is a statistical weight to measure the importance of a term in a collection of documents. The more usage a term has across documents, the less relevant it is to the particular document concerned. 

We would be computing the IDF of each token across all messages to find out their significance.

In [0]:
#finding the IDF and saving it the tf_idf column
idf = IDF(inputCol='c_vec',outputCol='tf_idf')

Now, in PySpark, we have a disadvantage of not being able to use classification labels as strings. So, we need to convert them into indexers to feed them to the model.

We would be using the StringIndexer to perform this operation.

In [0]:
#converts ham/spam to 0/1 format
ham_spam_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler

Finally, we are in the stage where we have all the columns required in the appropriate format. The final straw is in the conversion of all the columns into one dense vector to make it libsvm compatible.

In [0]:
#create the features column having the needed columns in a dense vector format
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

Now that we have the needed dataframe with two columns - features and class, we are ready to build our classification model. Here, we would be using the Naive Bayes' model for the same.

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
#creating a Naive Bayes model object with the default parameters
nb = NaiveBayes()

Finally, we would be performing all the operations in a sequence on the original dataset to make it memory efficient and less time consuming.

In [0]:
from pyspark.ml import Pipeline

In [0]:
#creating the pipeline object with the stages in the sequential manner
data_prep_pipe = Pipeline(stages=[ham_spam_numeric,tokenizer,stop_remove,countvec,idf,clean_up])

In [0]:
#Fitting the pipeline object with our data
cleaner = data_prep_pipe.fit(data)

In [0]:
#Transform the original dataset 
clean_data = cleaner.transform(data)

In [0]:
#select only the label and features column  
final_data = clean_data.select('label','features')
final_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

Model Building and training are done now on 80% of the dataset and it is tested against 20% of the dataset.

In [0]:
#test train split
train,test = final_data.randomSplit([0.8,0.2])

In [0]:
#train the Naive Bayes model
spam_detector = nb.fit(train)

In [0]:
#Predict on the test dataset
test_results = spam_detector.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,7,8...|[-792.38448252351...|[1.0,6.6562709905...| 0.0|
 0.0|(13424,[0,1,5,20,...|[-803.66953181862...|[1.0,1.3296035501...| 0.0|
 0.0|(13424,[0,1,14,18...|[-1373.0612011283...|[1.0,6.8483363421...| 0.0|
 0.0|(13424,[0,1,15,20...|[-676.94181415862...|[1.0,8.2292105524...| 0.0|
 0.0|(13424,[0,1,17,19...|[-806.30285256101...|[1.0,5.0408693534...| 0.0|
 0.0|(13424,[0,1,24,31...|[-343.07651460777...|[1.0,1.5546896402...| 0.0|
 0.0|(13424,[0,1,27,35...|[-1479.7481923014...|[0.99999999999998...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-988.59500667556...|[1.0,4.5220183940...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-1618.6251418899...|[1.0,5.5692797648...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-516.31084523058...|[1.0,1.3602106851...| 0.0|
 0.0|(13424,[0,2,4,10,...|[-1228.5329583926...|[1.0,6.4805499018...| 0.0|
 0.0|(13424,[0,2,4,25,...|[-440.78238607243...|[1.0,3.6971028161...| 0.0|
 0.0|(13424,[0,2,4,40,...|[-1593.9798208678...|[0.99961990007688...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-688.12935023614...|[1.0,7.2345025488...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-1341.9303396725...|[1.0,1.1581876407...| 0.0|
 0.0|(13424,[0,2,7,31,...|[-671.54148869120...|[1.0,2.9557336467...| 0.0|
 0.0|(13424,[0,2,11,13...|[-1398.6780646412...|[1.0,4.6058060702...| 0.0|
 0.0|(13424,[0,2,14,24...|[-431.20302497589...|[0.99999999999997...| 0.0|
 0.0|(13424,[0,2,20,60...|[-1231.7729839481...|[1.0,2.8380194435...| 0.0|
 0.0|(13424,[0,2,20,10...|[-383.94501479769...|[1.0,1.8518971207...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
#evaluating the prediction of the spam and ham on the test dataset
acc_eval = MulticlassClassificationEvaluator()

In [0]:
#computing the accuracy
acc = acc_eval.evaluate(test_results)
acc

Out[28]: 0.9194453777008199

Thus we can see how raw texts can be fed into mathematical machine learning models for classification. Since by using the default parameters of the Naive Bayes model, we have achieved a near perfect model with an accuracy of 93%, we can conclude that our model is good to go for the classification.